# Mixer - Simple Parameter

Author: Felix Hoffmann <br>
Date: 05/01/2024 <br>
Based on: Reduced Basis Methods for Partial Differential Equations - An Introduction

This Tutorial will extend the previous tutorial and will include how to rewrite the simulation to allow parameter investigation. 

---

We want to write a function that takes just the Parameters and returns the solution. Ideally, we want to avoid repeating computations. For example, we only need to read in the mesh once and can pass it on.

<font color='Red'> <b>ToDo:</b> </font> <br>
The following code will reinitialize the weak formulation each time. Switch the solver to ```PetSc.KSP()```and use the ```setOpertors```, ```update``` and ```assemble``` to avoid reinitializing the weak formulation in each step. 

---

First, we will do the offline computation. That means steps we only need to do once. 

In [1]:
### --- Imports --- ###
import numpy as np

import src.helper as helper
import dolfinx
import dolfinx.mesh as xmesh
import dolfinx.fem as xfem
import dolfinx.fem.petsc as xpetsc
import dolfinx.io as xio

import ufl

# Plotting 
import pyvista
import trame
import ipywidgets

In [2]:
def offlineComputation():
    # Perform all computations which are independent 
    #   of mu. THese can be performed offline. 
    #################################################
    
    import src.offlineHelper as offH
    import src.flowField.flowField as flowField

    # Read in the mesh
    fileName = "src/mesh/mixer.msh"
    mesh, cell_markers, facet_markers =  offH.readMesh(fileName)

    # Create FunctionSpace & Test-/Trialfunction & B-Field
    V, u, v = offH.createFunctionSpace(mesh)
    b = flowField.returnB(mesh)
    
    # Define and Perform Boundaries
    boundaries = offH.setUpBc()
    facet_tag = offH.tagBc(mesh, boundaries)
    offH.saveBc2Mesh(mesh, facet_tag)
    ds = ufl.Measure("ds", domain=mesh, subdomain_data=facet_tag) 

    return V, u, v, b, mesh, ds, facet_tag

In [3]:
def onlineComputation(mu, V, u, v, b, mesh, ds, facet_tag):
    # Perform all computations which are depending 
    #   on mu. These need to be performed after mu is feeded. 
    #########################################################
    
    import src.onlineHelper as onH
    
    # Define Weak Formulation
    FF = onH.defineWeakForm(mu, u, v, b)

    # Apply Boundary Conditions
    boundary_conditions = onH.evalBc(mu, mesh, V, v, ds, facet_tag)
    FF, bcs = onH.performBoundaryConditions(boundary_conditions, FF)
    
    ### Solve the Problem
    # Define LHS / RHS
    a = ufl.lhs(FF)
    L = ufl.rhs(FF)
    
    # Specify Solver
    problem = xpetsc.LinearProblem(a, L, bcs=bcs)
    
    # Solve the actual problem
    uh = problem.solve()

    return uh

In [4]:
# Setup Code
V, u, v, b, mesh, ds, facet_tag = offlineComputation()
calcPDE = lambda mu: onlineComputation(mu, V, u, v, b, mesh, ds, facet_tag)

Info    : Reading 'src/mesh/mixer.msh'...
Info    : 49 entities
Info    : 1655 nodes
Info    : 3308 elements
Info    : Done reading 'src/mesh/mixer.msh'


In [5]:
# calculate PDE
mu = np.array([4, 6, 2, 125])
uh = calcPDE(mu)

In [ ]:
# Plot Solution
helper.plotSolution(uh, V)

---

We now have created a function that returns the solution after feeding in a parameter $\mu$. Before we dive into parameter investigation, let's have a closer look at our solver. We want to investigate the saved time by doing the splitting into offline and online computation.

<font color='Red'> <b>ToDo:</b> </font> <br>
Compare speed between different solvers. Also compare the error between different solvers. 

In [6]:
import time
import tqdm

NN = 1000

# Offline and Online in each Step
startTimeOff = time.time()

for _ in range(NN):
    V, u, v, b, mesh, ds, facet_tag = offlineComputation()
    calcPDE = lambda mu: onlineComputation(mu, V, u, v, b, mesh, ds, facet_tag)

    mu = np.random.rand(4) * np.array([12, 12, 12, 200])
    uh = calcPDE(mu)

endTimeOff = time.time()

# ------------------------------------------------------------------------------------- #

# Seperation in Offline and Online
startTimeOn = time.time()

V, u, v, b, mesh, ds, facet_tag = offlineComputation()
calcPDE = lambda mu: onlineComputation(mu, V, u, v, b, mesh, ds, facet_tag)

for _ in range(NN):
    mu = np.random.rand(4) * np.array([12, 12, 12, 200])
    uh = calcPDE(mu)

endTimeOn = time.time()

Info    : Reading 'src/mesh/mixer.msh'...
Info    : 49 entities
Info    : 1655 nodes
Info    : 3308 elements
Info    : Done reading 'src/mesh/mixer.msh'
Info    : Reading 'src/mesh/mixer.msh'...
Info    : 49 entities
Info    : 1655 nodes
Info    : 3308 elements
Info    : Done reading 'src/mesh/mixer.msh'
Info    : Reading 'src/mesh/mixer.msh'...
Info    : 49 entities
Info    : 1655 nodes
Info    : 3308 elements
Info    : Done reading 'src/mesh/mixer.msh'
Info    : Reading 'src/mesh/mixer.msh'...
Info    : 49 entities
Info    : 1655 nodes
Info    : 3308 elements
Info    : Done reading 'src/mesh/mixer.msh'
Info    : Reading 'src/mesh/mixer.msh'...
Info    : 49 entities
Info    : 1655 nodes
Info    : 3308 elements
Info    : Done reading 'src/mesh/mixer.msh'
Info    : Reading 'src/mesh/mixer.msh'...
Info    : 49 entities
Info    : 1655 nodes
Info    : 3308 elements
Info    : Done reading 'src/mesh/mixer.msh'
Info    : Reading 'src/mesh/mixer.msh'...
Info    : 49 entities
Info    : 1655 nod

In [10]:
compTimeOff = endTimeOff - startTimeOff
compTimeOn  = endTimeOn - startTimeOn
print(f"Elapsed Time for Offline&Online in each step:\n\t {compTimeOff / NN}")
print(f"Elapsed Time for seperation into Offline & Online:\n\t {compTimeOn / NN}\n")
print(f"Saved Time: {np.round((compTimeOff - compTimeOn) / compTimeOn * 100,1)}%")

Elapsed Time for Offline&Online in each step:
	 1.7179015398025512
Elapsed Time for seperation into Offline & Online:
	 1.5081001043319702

Saved Time: 13.9%


So we see that splitting the offline and online computation saves us around $14\%$ of computational speed (on my machine). This doesn't sound much at first, but if your simulation runs for one hour then this is an additional 10min. 